# 강화학습 Reinforcement Learning 

예) 자율주행 자동차, 추천 시스템, 웹 페이지에 광고 배치하기 ...

Policy 정책 = 소프트웨어 에이전트의 행동 결정 알고리즘

로봇청소기 예시)
- 보상 : 30분동안 수집한 먼지의 양
- policy : 확률p만큼 전진 or (1-p) 확률로 좌우로 랜덤하게 회전(회전 각도는 -r ~ +r 랜덤일 수 있다) -> 무작위성 -> 확률적 정책 stochastic policy
- 확률 p, 각도의 범위 r - 두 개의 정책 파라미터 policy parameter의 가장 좋은 조합찾기
    -  이 때, policy space 정책 공간이 매우 커서 좋은 파라미터 조합을 찾기 힘든 경우가 많음    
    **Policy space 탐색 방법**
    1. Genetic algorithm 유전 알고리즘 : 1세대 정책 100개 랜덤 생성 -> 성능이 높은 20개 정책 선택 -> 선택된 정책에 자식 정책 4개씩 생산하게 함 -> 좋은 정책 찾을 때까지 반복
    2. Policy gradient 정책 그레이디언트 : 보상 gradient 를 평가 -> 높은 보상 방향인 그레이디언트로 파라미터값 수정 (=최적화 기법)    
    에이전트가 활동할 환경 마련(=OpenAI 짐) -> 텐서플로로 구현가능

### OpenAI 짐
에이전트가 활동할 환경을 마련해준다.
(실제 로봇을 이용해서 테스트하는데 생기는 시공간 제약을 극복가능)

# 신경망 정책
관측 input -> output 확률 추정 -> 추정된 확률에 따라 랜덤으로 output 선택

확률 기반 랜덤 선택 이유 : 새로운 output 을 exploring 탐험, 잘 할 수 있는 output(행동) exploiting 활용의 Balance 잡기 위해서임.

예를들어, 어느 베트남 식당을 갔을 때 쌀국수가 맛있다가 잘 할수있는 행동으로 계속 expoliting 선택된다면 더 맛있을 수 있는 분짜를 새로운 exploring을 하지 못하기 때문에 둘의 Balance를 잘잡는게 중요하다고 한다.   
이런 경우는 input이 환경에 대해 완전한 상태를 담고 있기 때문에, 과거의 input, output은 무시해도 된다.   
만약, 카트의 위치만 제공되고 현재 속도를 알아내야 한다면, 과거의 input도 고려해야 한다.

# 행동 평가
- 각 스텝마다 좋은 output(행동) 알고 싶다 -> (추정된 확률, 타깃 확률 사이) 크로스 엔트로피 최소화하는 훈련 가능
- 에이전트가 보상을 받을 때 어떤 행동 덕분인지 모름 -> 각 단계별 대가 return(할인계수를 적용한 보상을 총합)으로 행동을 평가한다 
- action advantage : 평균적으로 다른 행동과 비교해서 각 action이 좋은지 나쁜지 추정 -> 대가 정규화 -> 행동 이익이 음수:나쁜 행동, 행동 이익이 양수 : 좋은 행동

# 정책 Gradient
PG 알고리즘 : 높은 보상을 얻는(보상을 증가시켜주는) 그레이디언트를 따라 파라미터값 최적화    
PG 알고리즘 중 하나인 **REINFORCE 알고리즘**에 대하여
1. 여러 번 훈련, 더 높은 가능성을 가지게 하는 그레이디언트 계산
2. 에피소드 n번 실행 후 각 행동 이익 계산
3. 이익이 양수이면 미래에 선택될 가능성이 높도록 앞서 계산해둔 그레이디언트를 적용, 음수라면 음수 행동이 덜 선택되도록 반대 그레이디언트 적용
4. 모든 결과 그레이디언트 벡터 평균으로 경사 하강법 스텝 수행

다음은 tf.keras를 이용한 알고리즘 구현이다.

In [2]:
# 한 스텝용 함수
# good action 인 것으로 가정
# 손실, 그레이디언트 계산용
import tensorflow as tf
import numpy as np
import keras

loss_fn = keras.losses.binary_crossentropy

def play_one_step(env, obs, model, loss_fn_):
    with tf.GradientTape() as tape:
        # called a Model(return 왼쪽으로 갈 확률)
        left_proba=model(obs[np.newaxis]) 
        # 0~1 랜덤실수 샘플링, 샘플링값이 left_proba보다 큰가?
        # left_proba 확률에 의해 action=False가 됨. 0(왼쪽)
        # 1-left_proba 확률은 True. 1(오른쪽)
        action=(tf.random.uniform([1,1])>left_proba) # 결과로 True, False return
        # 오니쪽으로 이동할 타깃 확률 정의 = 1.0 - 왼쪽으로 갈 확률
        # action이 0이면 y_target 타깃 확률은 1이 된다.
        y_target=tf.constant([[1.]]) - tf.cast(action, tf.float32)
        # 손실함수(binary_crossentropy)로 손실계산
        loss=tf.reduce_mean(loss_fn(y_target,left_proba))
    # 모델 훈련가능 변수에 대한 손실의 gradient 계산값
    grads=tape.gradient(loss, model.trainable_variables)
    # 새로운 관측, 보상, 에피소드 종료 여부, 계산한 그레이디언트 return
    obs, reward, done, info = env.step(int(action[0,0].numpy()))
    return obs, reward, done, grads

상단의 계산 함수를 여러 에피소드를 플레이하고 전체 보상과 각 에피소드, 스텝의 그레이디언트를 반환해본다.

In [ ]:
def play_multiple_episodes(env, n_episodes, n_max_steps, model, loss_fn):
    all_rewards = [] # 전체 보상목록
    all_grads = [] # 전체 그레이디언트 목록
    for episode in range(n_episodes):
        current_rewards = [] # 한 에피소드의 보상목록
        current_grads = []
        obs = env.reset()
        for step in range(n_max_steps):
            obs, reward, done, grads = play_one_step(env, obs, model, loss_fn) # 한 스텝의 계산 적용
            current_rewards.append(reward)
            current_grads.append(grads)
            if done:
                break
        all_rewards.append(current_rewards)
        all_grads.append(current_grads)
    return all_rewards, all_grads

In [5]:
def discount_rewards(rewards, discount_rate):
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_rate
    return discounted

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate)
                              for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean) / reward_std 
            for discounted_rewards in all_discounted_rewards]      # 정규화 return
print("기대값 :",discount_rewards([10, 0, -50], discount_rate=0.8) )
print(discount_and_normalize_rewards([[10, 0, -50], [10, 20]], discount_rate=0.8)) # 정규화된 행동이익

기대값 : [-22 -40 -50]
[array([-0.28435071, -0.86597718, -1.18910299]), array([1.26665318, 1.0727777 ])]


첫번째 에피소드는 두 번째에 비해 나쁘므로 Bad action임
두번째 에피소드가 Good action으로 됨

In [9]:
n_iterations = 150           # 150번 훈련한다
n_episodes_per_update = 10
n_max_steps = 200
discount_rate = 0.95         # 할인계수

optimizer = keras.optimizers.Adam(learning_rate=0.01) # 최적화 기법 Adam
loss_fn = keras.losses.binary_crossentropy  # 손실함수

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(5, activation="elu", input_shape=[4]),
    keras.layers.Dense(1, activation="sigmoid"),
])
env = gym.make("CartPole-v1")
env.seed(42);

for iteration in range(n_iterations):
    all_rewards, all_grads = play_multiple_episodes(
        env, n_episodes_per_update, n_max_steps, model, loss_fn)
    total_rewards = sum(map(sum, all_rewards))                     # Not shown in the book
    print("\rIteration: {}, mean rewards: {:.1f}".format(          # Not shown
        iteration, total_rewards / n_episodes_per_update), end="") # Not shown
    all_final_rewards = discount_and_normalize_rewards(all_rewards,
                                                       discount_rate) # 정규화된 행동이익
    all_mean_grads = []
    for var_index in range(len(model.trainable_variables)):
        # 평균 gradient
        mean_grads = tf.reduce_mean(
            [final_reward * all_grads[episode_index][step][var_index]
             for episode_index, final_rewards in enumerate(all_final_rewards)
                 for step, final_reward in enumerate(final_rewards)], axis=0) # 보상과 그레이디언트를 계산한 평균값
        all_mean_grads.append(mean_grads)
    # 평균 gradient를 optimizer에 적용한다 -> 모델 훈련 가능 변수가 변경, 정책 향상 
    optimizer.apply_gradients(zip(all_mean_grads, model.trainable_variables))

env.close()

2023-04-17 18:17:59.180178: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'gym' is not defined

위의 방법은 sample efficiency 샘플 효율성이 좋지 못해 더 크고 복잡한 문제에 적용이 힘들다.    
아주 긴 시간 플레이해야 정책이 많이 개선된다.    

# 마르코프 결정 과정
독립적인 행동으로 이루어진거라면 마르코프 결정 과정으로 강화학습 모델링 가능
Stochastic process
1. 정해진 개수의 상태
2. 스텝마다 한 상태가 다른 상태로 랜덤하게 전이된다
3. 각 상태 전이 확률은 고정이다
4. 과거 상태는 상관없다
5. (상태, 전이할 상태) 쌍에만 의존한다

여기서 확률이 있고 가는 방향이 여러개라면 어느 방향으로 가야할지 랜덤으로 가게 되는데, 이 것을 '최적의 상태 가치'를 추정하는 방법으로 효율적으로 만들 수 있다.

단점    
- 에이전트가 초기에는 전이 확률을 모름
- 보상 모름
- 알려면 적어도 한 번 상태, 전이 해야함
- 신뢰성있는 추정하려면 여러 번 경험 해야함

에이전트(행동을 선택하는 자)를 위한 최적의 정책
**Q-value** (최적의 상태-행동 가치 state-action value를 추정 하는 알고리즘)    
: s에서 행동 a를 선택한뒤 평균 기대하는 미래 보상의 합    
에이전트가 최적으로 행동한다는 가정아래임    
사용 과정 1. Q-value 추정값 0으로 초기화 2. Q-value iteraction 알고리즘 사용하여 update

**Q-value 구현하기**    
$\pi^*(s)=argmaxQ^*(s,a)$    

In [1]:
transition_probabilities = [ # shape=[s, a, s']
        [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]],
        [[0.0, 1.0, 0.0], None, [0.0, 0.0, 1.0]],
        [None, [0.8, 0.1, 0.1], None]]
rewards = [ # shape=[s, a, s']
        [[+10, 0, 0], [0, 0, 0], [0, 0, 0]],
        [[0, 0, 0], [0, 0, 0], [0, 0, -50]],
        [[0, 0, 0], [+40, 0, 0], [0, 0, 0]]]
possible_actions = [[0, 1, 2], [0, 2], [1]]

Q_values = np.full((3, 3), -np.inf) # -np.inf for impossible actions
for state, actions in enumerate(possible_actions):
    Q_values[state, actions] = 0.0  # for all possible actions
    
gamma = 0.90  # the discount factor

history1 = [] # Not shown in the book (for the figure below)
for iteration in range(50):
    Q_prev = Q_values.copy()
    history1.append(Q_prev) # Not shown
    for s in range(3):
        for a in possible_actions[s]:
            Q_values[s, a] = np.sum([
                    transition_probabilities[s][a][sp]
                    * (rewards[s][a][sp] + gamma * np.max(Q_prev[sp]))
                for sp in range(3)])

history1 = np.array(history1) # Not shown

np.argmax(Q_values, axis=1)

In [ ]:
Q_values = np.full((3, 3), -np.inf) # -np.inf for impossible actions
for state, actions in enumerate(possible_actions):
    Q_values[state, actions] = 0.0  # for all possible actions
    
gamma = 0.95  # the discount factor

for iteration in range(50):
    Q_prev = Q_values.copy()
    for s in range(3):
        for a in possible_actions[s]:
            Q_values[s, a] = np.sum([
                    transition_probabilities[s][a][sp]
                    * (rewards[s][a][sp] + gamma * np.max(Q_prev[sp]))
                for sp in range(3)])
            
np.argmax(Q_values, axis=1)

# 시간차 학습 Temporal difference learning TD학습
에이전트가 MDP 일부 정보만 알고 있을 때 다루는 *가치 반복 알고리즘 변형*    
상태를 떠날때 얻는 당장의 보상 + 기대 보상 = 이동 평균    을 저장함

# Q-learning
확률, 보상을 초기에 모른 상태 + Q-value 반복 알고리즘 적용

- 플레이를 보고 점진적으로 Q-value 추정을 향상
- 탐욕적 정책(가장 높은 Q-value를 선택하는게 최적의 정책)
- 보상r + 미래 보상의 합 = 이동 평균 
- 미래 보상의 합 = 타깃 정책 이후 최적의 행동 가정, 상태 s'의 Q-value 추정 최댓값 선택